In [21]:
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
def get_return_rate(df, shift):
    return np.power(df/df.shift(shift), (1/shift))-np.ones(df.shape[0])

pce = pd.read_csv('../data/PCEPI.csv', index_col='DATE')
pce = pce.iloc[::12, :].PCEPI.pct_change()
pce.index.name = 'Date'

def adjust_for_inflation(close_df, close_col, return_df, return_col):
    df_inf = pd.merge(close_df, return_df, left_index=True, right_index=True).join(pce)
    data = [df_inf.iloc[0][close_col]]
    for idx, v in df_inf[1:].iterrows():
        i = len(data)
        data.append(data[i-1]*(1+v[return_col])*(1-v.PCEPI))
    return pd.Series(data, index=df_inf.index)

def tax_returns(returns, frequencies):
    data = {}
    for freq in frequencies:
        r = (returns-returns.shift(freq))
        r_tax = r*0.2
        total_r = (returns-r_tax).clip(lower=returns)
        data[f'{freq}Y'] = get_return_rate(total_r, freq)
    return pd.DataFrame(data, index=returns.index)

In [19]:
df_bond = pd.read_csv('../data/bonds.csv', index_col='Year')
df_bond['Total Return'] = df_bond['Total Return'].str.rstrip('%').astype(float)/100
df_bond.index.name = 'Date'
print(df_bond.describe())

       Total Return
count     44.000000
mean       0.075020
std        0.067397
min       -0.029200
25%        0.029400
50%        0.064200
75%        0.096750
max        0.326000


In [20]:
import yfinance as yf
df_com = yf.Ticker('^SPGSCI').history(start='1985-01-01', interval='1mo')
df_com = df_com.iloc[::12, :].Close.pct_change()
print(df_com.describe())

count    39.000000
mean      0.053392
std       0.236162
min      -0.447385
25%      -0.070433
50%       0.006524
75%       0.135794
max       0.526531
Name: Close, dtype: float64
